Đồ án cuối kỳ môn DataMinning

Giảng viên: thầy Lê Anh Cường

Thực hiện:



1.   51800585 - Nguyễn Hữu Nghĩa
2.   51800578 - Nguyễn Văn Long
3.   51800956 - Hồ Xuân Anh Vũ









In [132]:
import pandas as pd
import numpy as np

In [133]:
## Đọc và xử lý dữ liệu
moviesdata_df = pd.read_csv('drive/MyDrive/BigData/u.item', sep="|" ,encoding='latin-1', header=None)
moviesdata_df = moviesdata_df[[0,1,2]]
moviesdata_df.columns = ['movieId', 'title', 'date']
ratings=pd.read_csv('drive/MyDrive/BigData/u.data',sep="\t" , header=None ,names = ['userId', 'movieId', 'rate', 'timestamp'])

In [134]:
moviesdata_df.head(10)

,movieId,title,date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995
6,7,Twelve Monkeys (1995),01-Jan-1995
7,8,Babe (1995),01-Jan-1995
8,9,Dead Man Walking (1995),01-Jan-1995
9,10,Richard III (1995),22-Jan-1996


In [135]:
ratings.head(10)

,userId,movieId,rate,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [136]:
ratings_table=ratings.pivot_table(index='userId',columns='movieId',values='rate').fillna(0)
ratings_table.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
!pip install surprise


In [137]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [138]:
# read rating data
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId','movieId','rate']], reader)
 
# train SVD 
trainset, testset = train_test_split(data, test_size=.25)
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)
 
# Mean Square Error
print(accuracy.rmse(predictions))

RMSE: 0.9387
0.9387355261020836


In [152]:
from collections import defaultdict
# defaultdict trả về mặc định là list 
def get_top_n(predictions, userId, movies_df, ratings_df, n = 10):
    #1. Ánh xạ các dự đoán cho người dùng
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, true_r, est))

    #2. Sắp xếp dự đoán cho người dùng và lấy k dự đoán cao nhất 
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_n[uid] = user_ratings[: n ]
    
    #3. Những movie mà user đã đánh giá
    user_data = ratings_df[ratings_df.userId == (userId)]
    print('User {0} has already rated {1} movies.'.format(userId, user_data.shape[0]))

    
    #4. Predictions. 
    preds_df = pd.DataFrame([(id, pair[0],pair[1],pair[2]) for id, row in top_n.items() for pair in row],
                        columns=["userId" ,"movieId","truerate", "rat_pred"])
    
    preds_df["movieId"] = preds_df["movieId"].astype(str)
    #5. Return pred_user Top movie user được dự đoán là sẽ đánh giá cao
    pred_user = preds_df[preds_df["userId"] == (userId)]
            
    ratings["movieId"] = ratings["movieId"].astype(str)
    #6. Return hist_user Top movie user rate cao nhất 
    hist_user = ratings_df[ratings_df.userId == (userId) ].sort_values("rate", ascending = False)
    
    
    return hist_user, pred_user

In [153]:
hist_SVD, pred_SVD = get_top_n(predictions, movies_df = moviesdata_df, userId = 156, ratings_df = ratings)

User 156 has already rated 37 movies.


In [ ]:
hist_SVD

In [154]:
pred_SVD

,userId,movieId,truerate,rat_pred
6753,156,641,5.0,3.894219
6754,156,357,4.0,4.160881
6755,156,661,4.0,3.834740
6756,156,528,4.0,4.053248
6757,156,317,4.0,3.678756
6758,156,646,4.0,3.668042
6759,156,64,3.0,4.326104
6760,156,12,3.0,4.222241
6761,156,806,3.0,3.141103
